In [27]:
import os
from pyspark import SparkConf,SparkContext
from pyspark.sql import HiveContext,Row 
from pyspark.sql.types import *

In [28]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [96]:
d = sc.parallelize([("Alice",18),("Bob",20), ("Tom",40)])

In [97]:
d2 = sc.parallelize([("Bob",85), (None,80)])

In [98]:
schema1 = StructType([
  StructField("name", StringType(), True),
  StructField("age", IntegerType(), True)])

In [99]:
schema2 = StructType([
  StructField("name", StringType(), True),
  StructField("height", IntegerType(), True)])

In [100]:
df = sqlContext.createDataFrame(d, schema1)
df2 = sqlContext.createDataFrame(d2, schema2)

In [106]:
print df.collect()
print df2.collect()

[Row(name=u'Alice', age=18), Row(name=u'Bob', age=20), Row(name=u'Tom', age=40)]
[Row(name=u'Bob', height=85), Row(name=None, height=80)]


In [121]:
df.join(df2, df.name == df2.name, 'outer').collect()

[Row(name=u'Tom', age=40, name=None, height=None),
 Row(name=None, age=None, name=None, height=80),
 Row(name=u'Bob', age=20, name=u'Bob', height=85),
 Row(name=u'Alice', age=18, name=None, height=None)]

In [122]:
df.join(df2, 'name', 'outer').collect()

[Row(name=u'Tom', age=40, height=None),
 Row(name=None, age=None, height=80),
 Row(name=u'Bob', age=20, height=85),
 Row(name=u'Alice', age=18, height=None)]

In [125]:
df.join(df2, 'name').collect()

[Row(name=u'Bob', age=20, height=85)]

In [126]:
df.join(df2, 'name', 'inner').collect()

[Row(name=u'Bob', age=20, height=85)]

In [120]:
df.join(df2, 'name', 'left_outer').collect()

[Row(name=u'Tom', age=40, height=None),
 Row(name=u'Bob', age=20, height=85),
 Row(name=u'Alice', age=18, height=None)]

In [119]:
df.join(df2, 'name', 'right_outer').collect()

[Row(name=None, age=None, height=80), Row(name=u'Bob', age=20, height=85)]

In [118]:
df.join(df2, 'name', 'leftsemi').collect()

[Row(name=u'Bob', age=20)]

In [128]:
df.join(df2, 'name', 'leftsemi').select(df.name, df2.height).collect() #Error

AnalysisException: u'resolved attribute(s) height#126 missing from name#120,age#121 in operator !Project [name#120, height#126];;\n!Project [name#120, height#126]\n+- Project [name#120, age#121]\n   +- Join LeftSemi, (name#120 = name#125)\n      :- LogicalRDD [name#120, age#121]\n      +- LogicalRDD [name#125, height#126]\n'